<a href="https://colab.research.google.com/github/rishabhjain0267/pythonproject/blob/main/DataScience_UpdatedwithLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from langdetect import detect
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from textblob import TextBlob

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.data.path.append("/path/to/nltk_data")



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [ ]:
data=pd.read_csv("modi.csv")

In [ ]:
import string
data['Embedded_text'] = data['Embedded_text'].apply(lambda x: re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", x))
data['Embedded_text'] = data['Embedded_text'].apply(lambda x: ' '.join (c.replace("\\",'') for c in x.split(' ')  ))
data['Embedded_text'] = data['Embedded_text'].apply(lambda x: ' '.join (c.replace("\n",'') for c in x.split(' ')  ))
data['Embedded_text'] = data['Embedded_text'].apply(lambda x: ' '.join (c.replace("\t",'') for c in x.split(' ')  ))
data['Embedded_text'] = data['Embedded_text'].apply(lambda x: ' '.join (c.replace("&amp",'') for c in x.split(' ')  ))
data['Embedded_text'] = data['Embedded_text'].apply(lambda x: ' '.join (c.split('@',1)[0] for c in x.split(' ')  ))
data['Embedded_text'] = data['Embedded_text'].apply(lambda x: ' '.join (c.split('#',1)[0] for c in x.split(' ')  ))
data['Embedded_text'] = data['Embedded_text'].apply(lambda x: ' '.join (c.translate(str.maketrans('', '', string.punctuation)) for c in x.split(' ')  ))

In [ ]:
# download the pre-trained GloVe embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip -d glove.6B

--2023-04-27 12:07:33--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-04-27 12:07:33--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-04-27 12:07:33--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np

In [ ]:
!pip install Keras-Preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.6 MB/s eta 0:00:00


In [ ]:
from textblob import TextBlob
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

# Remove unnecessary tweets based on keywords or regular expressions
data = data[~data['Embedded_text'].str.contains('personal life|non-political|unrelated topics', case=False, regex=True)]

# Filter out non-English tweets
def detect_language(text):
    try:
        return detect(text)
    except:
        return None

data['language'] = data['Embedded_text'].apply(detect_language)
data = data[data['language'].isin(['en'])]

# Tokenize the tweets
def tokenize_text(text):
    pattern = r'\b\w\w+\b'
    tokens = word_tokenize(text.lower())
    filtered_tokens = [token for token in tokens if re.match(pattern, token)]
    return filtered_tokens

data['tokens'] = data['Embedded_text'].apply(tokenize_text)

# Remove stop words
stop_words = set(stopwords.words('english'))

def remove_stop_words(tokens):
    return [word for word in tokens if word not in stop_words]

data['tokens'] = data['tokens'].apply(remove_stop_words)

# Stem the words
ps = PorterStemmer()
wnl = WordNetLemmatizer()

def stem_text(tokens):
    return [ps.stem(word) for word in tokens]

def lemmatize_text(tokens):
    return [wnl.lemmatize(word) for word in tokens]

data['stemmed_tokens'] = data['tokens'].apply(stem_text)
data['lemmatized_tokens'] = data['tokens'].apply(lemmatize_text)

def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity




In [ ]:
data['sentiment'] = data['Embedded_text'].apply(get_sentiment)

# Train a sentiment analysis model using LSTM
max_length = 100 # max length of a tweet
vocab_size = 10000 # max number of words in the vocabulary

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(data['Embedded_text'])
sequences = tokenizer.texts_to_sequences(data['Embedded_text'])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(data['sentiment'].apply(lambda x: 1 if x >= 0 else 0))

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# load the GloVe embeddings into a dictionary
embeddings_index = {}
with open('glove.6B/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs



In [ ]:
# create a matrix of embeddings for the words in our vocabulary
embedding_dim = 100 # the dimension of the embedding space
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i >= vocab_size:
        break
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss=['binary_crossentropy'] , optimizer='adam', metrics=['accuracy'])
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          1000000   
                                                                 
 bidirectional (Bidirectiona  (None, 100, 256)         234496    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              394240    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 32)                8224      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,636,993
Trainable params: 636,993
Non-tr

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=5, patience=4)
rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)

model.fit(X_train, y_train, epochs=20, batch_size=1024, validation_split=0.1, callbacks=[es, rlrp])




Epoch 1/20
11/11 [==============================] - 146s 12s/step - loss: 0.4083 - accuracy: 0.8366 - val_loss: 0.2761 - val_accuracy: 0.9228 - lr: 0.0010
Epoch 2/20
11/11 [==============================] - 122s 11s/step - loss: 0.2944 - accuracy: 0.9148 - val_loss: 0.2717 - val_accuracy: 0.9228 - lr: 0.0010
Epoch 3/20
11/11 [==============================] - 119s 11s/step - loss: 0.2902 - accuracy: 0.9149 - val_loss: 0.2676 - val_accuracy: 0.9245 - lr: 0.0010
Epoch 4/20
11/11 [==============================] - 121s 11s/step - loss: 0.2865 - accuracy: 0.9151 - val_loss: 0.2619 - val_accuracy: 0.9236 - lr: 0.0010
Epoch 5/20
11/11 [==============================] - 130s 12s/step - loss: 0.2768 - accuracy: 0.9153 - val_loss: 0.2412 - val_accuracy: 0.9236 - lr: 0.0010
Epoch 6/20
11/11 [==============================] - 124s 11s/step - loss: 0.2600 - accuracy: 0.9154 - val_loss: 0.2318 - val_accuracy: 0.9236 - lr: 0.0010
Epoch 7/20
11/11 [==============================] - 118s 11s/step - lo

KeyboardInterrupt: ignored

In [ ]:
# evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {loss:.3f}')
print(f'Test accuracy: {accuracy*100:.2f}%')

In [ ]:
# Topic Modelling
vectorizer = CountVectorizer(tokenizer=tokenize_text, stop_words='english', min_df=5)
doc_term_matrix = vectorizer.fit_transform(data['Embedded_text'])
feature_names = vectorizer.get_feature_names()
LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(doc_term_matrix)

for index, topic in enumerate(LDA.components_):
    print(f'Top 10 words for Topic #{index}')
    topic_words = [feature_names[i] for i in topic.argsort()[-10:]]
    topic_words = [word for word in topic_words if len(word) > 2]
    print(topic_words)
